# Patterns of Kind Abstraction
Explanations of some of the ways we can achieve abstraction through type level programming.

- Funtional dependencies
- Associated types
- Type families

Let's define some types that facilitate the implementation of generic folding functions. We are using the 'Sum of Products' style here (as opposed to the 'Oragami' or 'Scrap Your Boilerplate' styles).

In [8]:
data List' a
  = Nil' | Cons' a (List' a)
  deriving (Show)

-- abstract representation types (could also represent a tree, for example)
data U = U                 
  deriving (Show)

data Choice a b = L a | R b   
  deriving (Show)

data Combo a b = Combo a b     
  deriving (Show)

type RList a = Choice U (Combo a (List' a))

-- converting between the two
fromL :: List' a -> RList a
fromL Nil'          = L U
fromL (Cons' x xs)  = R (Combo x xs)

toL :: RList a -> List' a
toL (L U)              = Nil'
toL (R (Combo x xs))   = (Cons' x xs)

-- a type for grouping converters together
data EP d r = EP {from :: (d -> r),
                 to :: (r -> d)}

-- in order to write generic functions, we first need to group our three representation types under one type.
-- We'll do this with a gadt
:ext GADTs
data TypeRep t where
  RUnit   :: TypeRep U
  RChoice :: TypeRep a -> TypeRep b 
                 -> TypeRep(Choice a b)
  RCombo  :: TypeRep a -> TypeRep b 
             -> TypeRep (Combo a b)
  RInt  :: TypeRep Int
  RType :: EP d r -> TypeRep r -> TypeRep d
  
-- we need a representation of RList made from TypeRep constructors:
rList :: TypeRep a -> TypeRep (List' a)
rList tr = RType (EP fromL toL) 
                 (RChoice RUnit  
                            (RCombo tr (rList tr)))
                            
-- now we define a generic function
gSize :: TypeRep a -> a -> Int

gSize RUnit  U  = 0

gSize (RChoice trA trB) (L a) 
  = gSize trA a
  
gSize (RChoice trA trB) (R b) 
  = gSize trB b
  
gSize (RCombo  trA trB) (Combo a b)
  = (gSize trA a) + (gSize trB b)
  
gSize RInt  _ = 1 

gSize (RType ep tr) t           
  = gSize tr (from ep t)
  
let aList = Cons' 1 (Cons' 3 (Cons' 7 Nil'))
gSize (rList RInt) aList

3

## Functional Dependencies
Instead of the `EP` container we used above, we could have used a multi-paramter type class with functional dependencies.

In [13]:
:ext FunctionalDependencies
:ext FlexibleInstances

class GenericFD d r | d -> r where
  from :: d -> r
  to   :: r -> d
  
instance GenericFD (List' a) (RList a) where
  from Nil'            = L U
  from (Cons' x xs)    = R (Combo x xs)
  to (L U)             = Nil'
  to (R (Combo x xs))  = (Cons' x xs)


from (Cons' "1" Nil')

R (Combo "1" Nil')

## Associated Type Synonyms
To get from functional dependencies to Associated types, we first observe that `GenericFD d r` doesn't actually need two parameters since `r` is uniquely determined by `d`.

In [14]:
:ext TypeFamilies

class GenericA d where
  type Rep d :: *
  
  fromA :: d -> Rep d
  toA   :: Rep d -> d

`Rep` is a type function (or 'type family', or 'associated type'). In contrast to functional dependencies, associated type synonyms make the type function explicit.

The generic functions `fromA` and `toA` are indexed against types that are themselves indexed by types! In this way, associated type synonyms extend type-classes by allowing for type-indexed behavior.

The type-class instance needs to specify a value for the type function `Rep`, that is, the instance mixes type functions with type-class functions.

In [16]:
instance GenericA (List' a) where
  type Rep (List' a) = RList a
  
  fromA Nil' = L U
  fromA (Cons' x xs) = R (Combo x xs)
  toA (L U) = Nil'
  toA (R (Combo x xs)) = Cons' x xs
  
fromA (Cons' 4 (Cons' 3 Nil'))

R (Combo 4 (Cons' 3 Nil'))

Here are the benefits of Associated types over Functional dependencies:

- The type functions are explicit, contrary to the implicit relations of FD
- Type functions allow us to reduce the number of parameters to a type class
- Type functions are more idiomatically functional than relational-style FD

## Type (synonym) Families
Associated types were subsumed by Type Families. Whereas Associated types are a special kind of type family where we attach a type function to a class, we can instead have top-level type families that are not associated with a class.

In [18]:
type family RepF d
type instance RepF (List' a) = RList a

A type family represents a set of types, and each instance represents a set member.

In [19]:
class GenericF d where
  fromF :: d -> RepF d
  toF   :: RepF d -> d
  
instance GenericF (List' a) where
  fromF (Nil') = L U
  fromF (Cons' x xs) = R (Combo x xs)
  toF (L U) = Nil'
  toF (R (Combo x xs)) = Cons' x xs
  
fromF $ Cons' 4 $ Cons' 2 Nil'

R (Combo 4 (Cons' 2 Nil'))

With associated types, we need to align the type function parameters with those of the type-class. Top-level type families don't have that restriction and are therefore more general than associated types. The fundamental difference between top­level and associated type families is in the scope of the type function.